# Generators
### Copyright Luca de Alfaro, 2020.  License: CC-BY-NC.

To understand what a generator is, let's start with a simple example.  
Assume we have a long list `words` of words, perhaps obtained by splitting a long document into the individual words.  The words in `words` can be either in uppercase, or in lowercase, or capitalized: we have no guarantees.

Our task is this: given an additional word `keyword`, we want to check whether `keyword` appears in the document, regardless of case.

One brute force way of doing this consists in creating a lowercase version of `words`, and check if `keyword`, once also put in lowercase, belongs to the list:

In [1]:
words = "I told you I do not need a PHONE I need a TOASTER".split()

def lowercase_list(words):
    """Returns the lowercase version of the list"""
    return [w.lower() for w in words]

def check_occurrence(words, keyword):
    return keyword.lower() in lowercase_list(words)

print(check_occurrence(words, "Phone"))

True


The problem with this approach is that we are building a whole new duplicate list, just for the purpose of checking membership.

One way of avoiding building the whole list is to inter-mingle the generation of the list and its use:

In [2]:
def check_occurrence(words, keyword):
    for w in words:
        if keyword == w.lower():
            return True
        return False

The problem with this is that it mixes the `for` loop in `check_occurrence` with the list transformation.
In this case, the list transformation is simple (it consists in a single call to `.lower()`), but if it were complex, it would be a much cleaner design to transform the list in a separate function.

Can we have a separate function, similar to `lowercase_list`, that avoids building a full list?


It turns out, yes.  Consider this function, which iterates over a list of words, and prints the lowercase version of each word:

In [3]:
def print_lowercase(words):
    for w in words:
        print(w.lower())

print_lowercase(words)

i
told
you
i
do
not
need
a
phone
i
need
a
toaster


The function converts each word to lowercase on the fly, and prints it, without ever building the list of all lowercase words.  Can we just, instead of printing the word, return it?  Then we could check membership without building a new list.  Does this work?

In [4]:
def return_lowercase(words):
    for w in words:
        return w.lower()

def check_occurrence(words, keyword):
    return keyword.lower() in return_lowercase(words)

print(check_occurrence(words, "Phone"))

False


What goes wrong?  The problem is that the `return` statement in line 3 above terminates the execution of the `for` loop.  So `return_lowercase` returns the first word `"i"` and terminates.  The check in line 6 then checks whether `"phone"` is a member of the list of characters `"i"`: if a string is used when a list is expected, the list of characters is used.
This is not true, and the check returns `False`.

Fundamentally, what goes wrong is that `return` does not allow us to return the lowercase words one by one, because it stops at the first word.  
We need a version of `return` that gives the result back, but "keeps going" in the loop.  This version of `return` is called `yield`.  Let's try it.

In [5]:
def yield_lowercase(words):
    for w in words:
        yield w.lower()

def check_occurrence(words, keyword):
    return keyword.lower() in yield_lowercase(words)

print(check_occurrence(words, "Phone"))

True


This works!

And the function `yield_lowercase` is called a generator, because it generates a list -- without ever having to store one.

The implementation of the `yield` statement is sophisticated, because it needs essentially to pause execution to return the value, keeping track of the place inside the function, so that execution can be restarted when the next value is needed.  But to understand how to use it, think of `yield` as a version of `print` that, instead of printing, returns the value, and then just like `print`, keeps going.

In [6]:
#@title Let's define a test helper.

def check_equal(x, y, msg=None):
    if x != y:
        if msg is None:
            print("Error:")
        else:
            print("Error in", msg, ":")
        print("    Your answer was:", x)
        print("    Correct answer: ", y)
    assert x == y, "%r and %r are different" % (x, y)

## Generators Are Infinitely Useful

So what can generators do?

One thing, as we discovered, is return a variant of a list without every fully building the variant as a list.  This is important, because often we have big data structures in memory, and we want to be able to operate on them without having to duplicate them when we need something else.

The most interesting use of generators, however, is to create on the fly -- without ever storing them -- structures that can be very large.

In particular, one of the fun facts about generators is that they provide a _finite_ representation for _infinite_ things.  Here is an iterator on even numbers:

In [7]:
def even_numbers():
    i = 0
    while True:
        yield 2 * i
        i += 1

for n in even_numbers():
    print("This is even:", n)
    if n > 7:
        break

This is even: 0
This is even: 2
This is even: 4
This is even: 6
This is even: 8


We stopped at 7, but `even_numbers` would have been quite happy to keep going.

Here's an iterator that produces all numbers that are not divisible by 2, 3, or 5.  Note how `yield` does not need to appear directly in a loop: it can appear inside if-then-else statements and anything else.

In [8]:
def not_div_235():
    i = 0
    while True:
        if (i % 2) * (i % 3) * (i % 5) > 0:
            yield i
        i += 1

for n in not_div_235():
    print(n)
    if n > 20:
        break

1
7
11
13
17
19
23


**Exercise:** Build a generator that returns the [Fibonacci numbers](https://en.wikipedia.org/wiki/Fibonacci_number).

In [9]:
#@title Exercise: implement a Fibonacci number generator

def fibonacci_generator():
    """Generates all Fibonacci numbers."""
    ### YOUR SOLUTION HERE
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a + b

In [10]:
# Tests 10 points: Fibonacci numbers

r = []
for n in fibonacci_generator():
    r.append(n)
    if n > 100:
        break
check_equal(r, [0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144])


**Exercise:** Build a generator that returns all the prime numbers.  The idea is to loop over all positive integers, test each one to see if it is prime, and if it is, `yield` it.

In [11]:
#@title Exercise: implement a prime number generator

# My solution is simple and not particularly optimized,
# and it is 12 lines long.

def prime_number_generator():
    """This generator returns all prime numbers."""
    ### YOUR SOLUTION HERE
    yield 2
    primes = [2]
    num = 3
    while True:
        is_prime = all(num % p != 0 for p in primes)
        if is_prime:
            primes.append(num)
            yield num
        num += 2

Let us use our generator:

In [12]:
i = 0
for n in prime_number_generator():
    print(n)
    i += 1
    if i == 10:
        break

2
3
5
7
11
13
17
19
23
29


In [13]:
# Tests 10 points: `prime_number_generator`



for n in prime_number_generator():
    if n == 33:
        raise Exception()
    elif n > 37:
        break


## Generators That Know When To Stop

Sometimes, while running a generator, you realize that there's nothing more that needs to be `yield`-ed.  In that case, you can terminate the execution with a `return`.   Let's write a generator that takes a list of words, and iterates over the words until a given stop-word is reached, or until the end of the list, whichever happens first.

In [14]:
def words_until_stop(stop_word, words):
    for w in words:
        if w == stop_word:
            # We stop the iteration.
            return
        else:
            yield w

Note that the function `words_until_stop` has two ways of terminating.  One is when the stop word is met, and the `return` executed.  The other is when the `for` loop runs to completion.

In [15]:
words = "I like to eat pears far more than apples".split()

for w in words_until_stop("pears", words):
    print(w)

I
like
to
eat


## Iterating Over Permutations

Let us now write an iterator that, given a list, returns all the permutations of elements in the list.  The [itertools module](https://docs.python.org/3.7/library/itertools.html) contains such an iterator, but it is rather instructive to write it ourselves.

Recall that a permutation is a reordering.  So if you have a list

    [1, 2, 3]

its six permutations are:

    [1, 2, 3]
    [1, 3, 2]
    [2, 1, 3]
    [2, 3, 1]
    [3, 1, 2]
    [3, 2, 1]

To generate all permutations, we decompose the problem.  For a list $l$, let $P(l)$ be its set of permutations.  For a set of lists $C$, and for a single list $s$, denote with

$$
s +^* C = \{s + c \mid c \in C \}
$$

the set obtaining by the concatenation of $s$ with every element of $C$.
For elements $x_1, \ldots, x_n$, we have:

$$
\begin{align*}
P([x_1, \ldots, x_n]) = \{ \, & [x_1] +^* P([x_2, \ldots, x_n]),\\
& [x_2] +^* P([x_1, x_3, \ldots, x_n]), \\
& \cdots \\
& [x_k] +^* P([x_1, \ldots, x_{k-1}, x_{k+1}, \ldots, x_n]),\\
& \ldots \\
& [x_n] +^* P([x_1, \ldots, x_{n-1}]) \, \} \; .
\end{align*}
$$

In other words, given a list $l$, we can iterate on it, choosing in turn an element $x$.  We can then compute the permutations of $l$ by concatenating $x$ with the permutations of the list $l'$ that results from removing $x$ from $l$.

In [16]:
def permute(elements):
    """Yields all the permutations of iterable, one by one."""
    if len(elements) == 0:
        yield []
    else:
        for i, x in enumerate(elements):
            # We separate elements into x, and into remainder, that
            # consists of all elements minus x.
            remainder = elements[:i] + elements[i+1:]
            for p in permute(remainder):
                yield [x] + p

In [17]:
l = [1, 2, 3]
for ll in permute(l):
    print(ll)

check_equal(len(list(permute([1, 2, 3, 4]))), 24)

[1, 2, 3]
[1, 3, 2]
[2, 1, 3]
[2, 3, 1]
[3, 1, 2]
[3, 2, 1]


### Iterating Over Combinations

Given a list containing distinct elements, assume we want to iterate over all pairs of elements of the list.
This is easy to do:

In [18]:
def iterate_pairs(elements):
    for i, x in enumerate(elements):
        for y in elements[i + 1:]:
            yield {x, y}

In [19]:
mylist = [1, 2, 3, 4]
for p in iterate_pairs(mylist):
    print(p)

{1, 2}
{1, 3}
{1, 4}
{2, 3}
{2, 4}
{3, 4}


What if I asked you to iterate over triples of elements?  
You could of course use three nested for loops, instead of just two.
But at some point, it becomes easier to write the general solution.

Consider a list $L = [x_1, x_2, \ldots, x_n]$ consisting of all distinct elements, and an integer $k$.
Let $C(k, L)$ be the set of subsets of $L$ containing $k$ elements.
$C(k, L)$ is also known as the _combinations of $L$ in groups of $k$_.

We will write a function `combinations(k, L)` that computes $C(k, L)$.
To do so, we will use recursion, and write $C(k, L)$ in terms of $C(k-1, \ldots)$.
That is, to generate the subsets of length $k$, we will rely on a recursive call (a call to $C$ itself) to generate the subsets of length $k-1$.
To write the recursion, let $\cup^*$ be the operator taking the union of a set with each set in a set:

$$
A \cup^* B = \{a \cup b \mid b \in B\} \; .
$$

There are three cases, depending on the relation between the size $k$ of the subset we want, and the number $n = |L|$ of elements in $L$.

1. If $k = n$, we can just return a set containing $L$.

2. If $k > n$, then obviously $C(k, L) = \emptyset$, where $\emptyset$ denotes the empty set, as we cannot obtain a subset bigger than the set itself.

3. If $k < n$, to generate the $k$-subsets of a list $L$, you pick each element $x$ of $L$ in turn, you consider the list $L'$ of elements following $x$ in $L$, and you return the union of $x$, and of a $k-1$-subset of $L'$.
For those who like the precision of formulas:

$$
\begin{align*}
C(k, [x_1, \ldots, x_n]) = \{ \, & \{x_1\} \cup^* C(k - 1, [x_2, \ldots, x_n]),\\
& \{x_2\} \cup^* C(k-1, [x_3, \ldots, x_n]), \\
& \cdots \\
& \{x_i\} \cup^* C(k-1, [x_{i+1}, \ldots, x_n]),\\
& \cdots \\
& \{x_n\} \cup^* C(k-1, \emptyset) \, \} \; .
\end{align*}
$$



**Exercise:** Implement a function `combinations(k, elements)` implementing the function $C$ above, that is, returning the set of $k$-elements subsets of `elements`.

Hint: the difficulty is not in writing the recursion, but in deciding when NOT to recur, and rather, yield a result or the absence of results.

In [20]:
#@title Exercise: Generating Combinations

def combinations(k, elements):
    assert isinstance(elements, list)
    # This can be done in 9 lines of code, and possibly fewer.
    ### YOUR SOLUTION HERE
    assert isinstance(elements, list)
    if k == 0:
        return [set()]
    if k > len(elements):
        return []
    if k == len(elements):
        return [set(elements)]
    without_first = combinations(k, elements[1:])
    with_first = [set([elements[0]]) | combo for combo in combinations(k - 1, elements[1:])]
    return with_first + without_first

In [21]:
# Tests 5 points: Basic tests for Combination Generation

# Let us start from some base cases.
L = [1, 2, 3, 4, 5]

# There are no combinations of 5 elements in groups of 6.
n = 0
for c in combinations(6, L):
    n += 1
check_equal(n, 0)


# There is only one combination of 5 elements in groups of 5: the set itself.
n = 0
for c in combinations(5, L):
    check_equal(c, set(L))
    n += 1
check_equal(n, 1)


In [22]:
# Tests 5 points: Basic tests 2 for Combination Generation
L = [1, 2, 3, 4, 5]

# There is only one combination of 5 elements in groups of 0: the empty set.
n = 0
for c in combinations(0, L):
    check_equal(c, set())
    n += 1
check_equal(n, 1)


In [23]:
# Tests 5 points: Basic tests 3 for Combination Generation
L = [1, 2, 3, 4, 5]

# There is only one combination of 5 elements in groups of 5: the set itself.
n = 0
for c in combinations(5, L):
    check_equal(c, set(L))
    n += 1
check_equal(n, 1)


In [24]:
# Tests 10 points: Normal tests for Combination Generator

L = [1, 3, 4, 2, 5]
c2 = combinations(2, L)
c3 = combinations(3, L)
n = 0
ok = False
for c in c2:
    n += 1
    check_equal(len(c), 2)
    if c == {3, 2}:
        ok = True
check_equal(n, 10, msg="n")
check_equal(ok, True, msg="ok")

n = 0
ok = False
for c in c3:
    check_equal(len(c), 3)
    n += 1
    if c == {2, 3, 5}:
        ok = True
check_equal(n, 10)
check_equal(ok, True)


### Iterating Over Subsets

Given a set, write an iterator that returns all subsets of the set.  

Hint: decompose the problem.  Given a set $S$, let $x \in S$ and $S' = S \setminus \{x\}$.  Denoting with $P(S)$ (the _powerset_ of $S$) the set of subsets of $S$, we have:

$$
P(S) = P(S') \cup \{T \cup \{x\} \mid T \in P(S') \} \; .
$$

In words, if you select an element of $x$, and let $S' = S \setminus \{x\}$, then the subsets of $S$ are the union of:

* the subsets of $S'$,
* the subsets of $S'$, with $x$ added to them.

This enables you to reduce the problem of computing the subsets of $S$, to that of computing the subsets of the smaller set $S'$.

In [25]:
#@title Exercise: implement the `subsets` function for sets

def subsets(s):
    """Given a set s, yield all the subsets of s,
    including s itself and the empty set."""
    ### YOUR SOLUTION HERE
    if not isinstance(s, set):
        s = set(s)
    if not s:
        yield set()
    else:
        x = next(iter(s))
        remaining = s - {x}
        for subset in subsets(remaining):
            yield subset
            yield subset | {x}

In [26]:
# Tests 10 points: `subsets`

s = set([1, 2, 3])
for t in subsets(s):
    print(t)

check_equal(len(list(subsets(s))), 8)


set()
{1}
{2}
{1, 2}
{3}
{1, 3}
{2, 3}
{1, 2, 3}
